In this benchmark, we compare throughput (MB/s) while reading a single ROOT file from different (geographical) locations, using different number of local workers.

### Purdue EOS, FUSE mounted

In [ ]:
import os, sys
sys.path.append(os.getcwd()+"/../af_benchmark")
from benchmark import Benchmark, run_benchmark
import numpy as np
import tqdm
import time
from dask_gateway import Gateway
%env XRD_REQUESTTIMEOUT 300

nworkers_options = [1,2,4,8,16]
col_num_options = [30,120,240,300]
# nworkers_options = [8]
# col_num_options = [1200]

b = Benchmark("config_3.2.yaml")
for nworkers in nworkers_options:
    for col_num in tqdm.tqdm(col_num_options):
        b.config["executor"]["n_workers"] = int(nworkers)
        b.config["processor"]["columns"] = int(col_num)
        b.reset(keep_cluster=True, reset_workers=True)
        b.run()
        b.update_report()

report = b.report_df

env: XRD_REQUESTTIMEOUT=300


2024-03-08 09:13:04,747 - distributed.preloading - INFO - Creating preload: import sys; sys.path.insert(0, '/home/spiperov/purdue-af-demo-may2023/');
2024-03-08 09:13:04,748 - distributed.utils - INFO - Reload module tmpah7fy39u from .py file
2024-03-08 09:13:04,748 - distributed.preloading - INFO - Import preload module: /tmp/tmpah7fy39u.py
2024-03-08 09:13:04,753 - distributed.preloading - INFO - Creating preload: import sys; sys.path.insert(0, '/home/spiperov/purdue-af-demo-may2023/');
2024-03-08 09:13:04,753 - distributed.utils - INFO - Reload module tmpij2gz_bk from .py file
2024-03-08 09:13:04,754 - distributed.preloading - INFO - Import preload module: /tmp/tmpij2gz_bk.py
2024-03-08 09:13:04,774 - distributed.preloading - INFO - Creating preload: import sys; sys.path.insert(0, '/home/spiperov/purdue-af-demo-may2023/');
2024-03-08 09:13:04,775 - distributed.utils - INFO - Reload module tmp597z0edi from .py file
2024-03-08 09:13:04,775 - distributed.preloading - INFO - Import prel

In [ ]:
import matplotlib.pyplot as plt

report["MB/s (compressed)"] = report.compressed_bytes/report.process_columns/(1024*1024)

report_by_ncols = report.groupby('n_columns_read')

plt.figure(figsize=(8, 6))
for ncols, group in report_by_ncols:
    plt.plot(group.n_workers, group["MB/s (compressed)"], label=f"{ncols} columns")

plt.xlabel('# workers')
plt.ylabel('MB/s')
plt.legend()
plt.xlim(0, report.n_workers.max()+5)
plt.ylim(0, report['MB/s (compressed)'].max() + 5)
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
for ncols, group in report_by_ncols:
    plt.plot(group.n_workers, group.process_columns, label=f"{ncols} columns")

plt.xlabel('# workers')
plt.ylabel('Time (s)')
plt.legend()
plt.xlim(0, report.n_workers.max()+5)
plt.ylim(0, report.run.max() + 5)
plt.show()